In [1]:
import numpy as np
import random
import copy

BoardSize = 5

def initializeBoard():
    return [[' ' for _ in range(BoardSize)] for _ in range(BoardSize)]

In [2]:
def checkRow(board, symbol):
    for i in range(BoardSize):
        colFlag = True
        for j in range(BoardSize):
            colFlag = colFlag and (board[i][j]==symbol)
        if(colFlag):
            return True
    return False

In [3]:
def checkColumn(board, symbol):
    for j in range(BoardSize):
        rowFlag = True
        for i in range(BoardSize):
            rowFlag = rowFlag and (board[i][j]==symbol)
        if(rowFlag):
            return True
    return False

In [4]:
def checkDiagonals(board, symbol):
    diag1Flag,diag2Flag = True, True
    for i in range(BoardSize):
        diag1Flag = diag1Flag and (board[i][i]==symbol)
        diag2Flag = diag2Flag and (board[BoardSize-1-i][i]==symbol)
    return (diag1Flag or diag2Flag)

In [5]:
def isGameOver(board, symbol):
    flag = (board==-1)
    flag = flag or checkRow(board,symbol) or checkColumn(board, symbol) or checkDiagonals(board, symbol)
    flag = flag or (' ' not in np.array(board).flatten())
    return flag

In [6]:
def getLegalMoves(boardState, symbol):
    
    legalMoves = []
    for i in range(len(boardState[0])):
        for j in range(len(boardState[0])):
            if(boardState[i][j] == ' '):
                tempBoard = copy.deepcopy(boardState)
                tempBoard[i][j]=symbol
                legalMoves.append(tempBoard)
    return legalMoves

In [7]:
def getFeatures(board, symbol1, symbol2):
    x = np.zeros(2*BoardSize+1)
    x[0] = 1
    for i in range(BoardSize):
        cntSymbol1,cntSymbol2 = 0,0
        for j in range(BoardSize):
            if(board[i][j]==symbol1):
                cntSymbol1+=1
            elif(board[i][j]==symbol2):
                cntSymbol2+=1
        if(cntSymbol1>=1):
            x[cntSymbol1]+=1
        if(cntSymbol2>=1):
            x[cntSymbol2+BoardSize]+=1
    
    for j in range(BoardSize):
        cntSymbol1,cntSymbol2 = 0,0
        for i in range(BoardSize):
            if(board[i][j]==symbol1):
                cntSymbol1+=1
            elif(board[i][j]==symbol2):
                cntSymbol2+=1
        if(cntSymbol1>=1):
            x[cntSymbol1]+=1
        if(cntSymbol2>=1):
            x[cntSymbol2+BoardSize]+=1
            
    return x

In [8]:
def printBoard(board):
    for i in range(BoardSize):
        for j in range(BoardSize):
            print(board[i][j],end='|')
        print()
        for _ in range(2*BoardSize):
            print("-",end='')
        print()

In [9]:
def getNonFinalBoardScore(weightVector,featureVector):
    weightVector = np.array(weightVector).reshape((len(weightVector),1))
    featureVector = np.array(featureVector).reshape((len(featureVector),1))
    boardScore = np.dot(weightVector.T,featureVector)
    return(boardScore[0][0])

In [10]:
def chooseMove(board,symbol1,symbol2,weightVector):

    legalMoves = getLegalMoves(board,symbol1)
    legalMoveScores = [getNonFinalBoardScore(weightVector,
                                             getFeatures(i,symbol1,symbol2)) for i in legalMoves]
    return legalMoves[np.argmax(legalMoveScores)]

In [11]:
def chooseRandomMove(board,symbol):
    legalMoves = getLegalMoves(board,symbol)
    return random.choice(legalMoves)

In [12]:
def getGameHistory(symbols,weightVector1,weightVector2,board):
    gameHistory = []
    gameStatusFlag = True
    
    tempBoard = copy.deepcopy(board)
    while(gameStatusFlag):
        tempBoard = chooseMove(tempBoard,symbols[0],symbols[1],weightVector1)
        gameHistory.append(tempBoard)
        gameStatusFlag = not isGameOver(tempBoard,symbols[0])
        if(gameStatusFlag == False):
            break
        tempBoard = chooseRandomMove(tempBoard,symbols[1])
        gameHistory.append(tempBoard)
        gameStatusFlag =  not isGameOver(tempBoard,symbols[1])
    return gameHistory

In [13]:
def getFinalBoardScore(board,symbol1,symbol2):
    score = 0
    if(checkRow(board,symbol1) or checkColumn(board, symbol1) or checkDiagonals(board, symbol1)):
        score = 100
    elif(checkRow(board,symbol2) or checkColumn(board, symbol2) or checkDiagonals(board, symbol2)):
        score = -100
    
    return score

In [14]:
def getTrainingSamples(weightVector,symbol1,symbol2,gameHistory):

    trainingData=[]
    for i in range(len(gameHistory)-1):
        featureVector = getFeatures(gameHistory[i+1],symbol1,symbol2)
        trainingData.append([featureVector,getNonFinalBoardScore(weightVector,featureVector)])
    trainingData.append([getFeatures(gameHistory[-1],symbol1,symbol2),
        getFinalBoardScore(gameHistory[-1],symbol1,symbol2)])
    return trainingData

In [15]:
def getGameStatusCount(symbol1,symbol2,gameStatusCount,gameHistory):
    finalScore = getFinalBoardScore(gameHistory[-1],symbol1,symbol2)
    if(finalScore == 100):
        gameStatusCount[0] += 1
    elif(finalScore == -100):
        gameStatusCount[1] += 1
    else:
        gameStatusCount[2] += 1
    return gameStatusCount

In [16]:
def LMSRule(weightVector,trainData,lRate=0.01):
    for t in trainData:
        vTrainBoardState = t[1]
        vHatBoardState = getNonFinalBoardScore(weightVector,t[0])
        weightVector = weightVector + (lRate * (vTrainBoardState - vHatBoardState) * np.array(t[0]))
    return weightVector

In [17]:
symbols = ('X','O')
numTrainingExamples = 1000

In [18]:
weightVectors = [np.random.rand(2*BoardSize+1),np.random.rand(2*BoardSize+1)]
gameStatusCount = [0,0,0]

for _ in range(numTrainingExamples):
    initialBoardState = initializeBoard()
    gameHistory = getGameHistory(symbols,weightVectors[0],weightVectors[1],initialBoardState)
    trainDataPlayer1 = getTrainingSamples(weightVectors[0],symbols[0],symbols[1],gameHistory)
    trainDataPlayer2 = getTrainingSamples(weightVectors[1],symbols[1],symbols[0],gameHistory)
    gameStatusCount = getGameStatusCount(symbols[0],symbols[1],gameStatusCount,gameHistory)
    weightVectors = [LMSRule(weightVectors[0],trainDataPlayer1),LMSRule(weightVectors[1],trainDataPlayer1)]


print("\nTraining Results: (" + "#Games Player-1 Wins = " + str(gameStatusCount[0]) +
    ", #Games Player-2 Wins = " + str(gameStatusCount[1]) + ", Games Drawn = " + str(gameStatusCount[2]) +
    ")\n")

learntWeight =  list(np.mean(np.array([weightVectors[0],weightVectors[1]]),axis = 0))
print("Final Learnt Weight Vector: \n"+ str(learntWeight))


Training Results: (#Games Player-1 Wins = 694, #Games Player-2 Wins = 9, Games Drawn = 297)

Final Learnt Weight Vector: 
[2.5648765977993193, 10.281926025480761, 3.783432641850567, -6.604745701224736, -15.939539823414716, 29.75365598062408, -1.8271619283249207, -2.813537330879281, 5.673356371949179, 9.736262815606622, 0.23887720551331232]


In [21]:
print("\nStart Playing with Computer?(y/n)")
ans = input() 
while(ans == "y"):
    boardState = initializeBoard()
    gameStatusFlag = True
    gameHistory = []
    print("Choose symbol: X or O")
    humanSymbol = input()
    if(humanSymbol=='X'):
        computerSymbol = 'O'
    else:
        computerSymbol = 'X'
    print("\nGame begins!!!\n")
    
    while(gameStatusFlag):
        
        if(humanSymbol=='X'):
            print('Your Turn:\n')
            print('Enter Row number (0-',BoardSize-1,")")
            x = int(input())
            print('Enter Column number (0-',BoardSize-1,")")
            y = int(input())
            
            boardState[x][y] = humanSymbol
            printBoard(boardState)
            gameHistory.append(boardState)
            gameStatusFlag =  not isGameOver(boardState,humanSymbol)
            if(gameStatusFlag == False):
                break
            boardState = chooseMove(boardState,computerSymbol,humanSymbol,learntWeight)
            print('Computers\'s Turn:\n')
            printBoard(boardState)
            gameHistory.append(boardState)
            gameStatusFlag = not isGameOver(boardState,computerSymbol)
            
        else:
            boardState = chooseMove(boardState,computerSymbol,humanSymbol,learntWeight)
            print('Computers\'s Turn:\n')
            printBoard(boardState)
            gameHistory.append(boardState)
            gameStatusFlag = not isGameOver(boardState,computerSymbol)
            if(gameStatusFlag == False):
                break
            print('Your Turn:\n')
            print("Enter Row number (0-",BoardSize-1,")")
            x = int(input())
            print('Enter Column number (0-',BoardSize-1,")")
            y = int(input())

            boardState[x][y] = humanSymbol
            printBoard(boardState)
            gameHistory.append(boardState)
            gameStatusFlag =  not isGameOver(boardState,humanSymbol)
    
    if(isGameOver(boardState,computerSymbol)):
        print("Computer Wins!!!")
    elif(isGameOver(boardState,humanSymbol)):
        print("You Win!!!")
    else:
        print("Game is drawn!!!")
    print("Do you want to play another game?(y/n).")
    ans = input()
    if(ans != 'y'):
        break


Start Playing with Computer?(y/n)
y
Choose symbol: X or O
O

Game begins!!!

Computers's Turn:

X| | | | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
Your Turn:

Enter Row number (0- 4 )
1
Enter Column number (0- 4 )
1
X| | | | |
----------
 |O| | | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
Computers's Turn:

X| | | | |
----------
 |O|X| | |
----------
 | | | | |
----------
 | | | | |
----------
 | | | | |
----------
Your Turn:

Enter Row number (0- 4 )
2
Enter Column number (0- 4 )
1
X| | | | |
----------
 |O|X| | |
----------
 |O| | | |
----------
 | | | | |
----------
 | | | | |
----------
Computers's Turn:

X| | | | |
----------
 |O|X| | |
----------
 |O| |X| |
----------
 | | | | |
----------
 | | | | |
----------
Your Turn:

Enter Row number (0- 4 )
0
Enter Column number (0- 4 )
1
X|O| | | |
----------
 |O|X| | |
----------
 |O| |X| |
----------
 | | | | |
----------
 | | | | |
---------